In [20]:
from docx import Document
import re
import os
# import openpyxl
from datetime import datetime
import pprint
import pandas as pd



transcripts_folder_path = os.getcwd()+'/raw_transcripts/'
time_format = "%H:%M:%S"
pretty_printer = pprint.PrettyPrinter(width=40, compact=True)
speaker_time_dict = {}

In [21]:
def match_pattern(pattern, text):
    # pattern = r"\['(?:\d+:)?\d+:\d+'\]"

    if re.search(pattern, text):
        return True
    else:
        return False

In [22]:
def find_doc_files_in_current_folder() -> list:
    """Find the docx file in the current folder
    It ignores the temperory files which are getting opened
    when files are opened '~'
    Returns:
        list: list of the files name as str
    """

    doc_files = []

    for filename in os.listdir(transcripts_folder_path):
        # filename.startswith("~") -> ignores the temprory files when that words make when file is open
        if filename.endswith(".docx") and not filename.startswith("~"):
            doc_files.append(filename)

    return doc_files

In [23]:
def calculate_time_delta(time1, time2):
    time1_parts = list(map(int, time1.split(':')))
    time2_parts = list(map(int, time2.split(':')))

    while len(time1_parts) < 3:
        time1_parts.insert(0, 0)
    while len(time2_parts) < 3:
        time2_parts.insert(0, 0)

    hours_diff = time2_parts[0] - time1_parts[0]
    minutes_diff = time2_parts[1] - time1_parts[1]
    seconds_diff = time2_parts[2] - time1_parts[2]

    total_seconds = hours_diff * 3600 + minutes_diff * 60 + seconds_diff
    return total_seconds

### Where I have the issue is to find the duration of each speaker

In [34]:
list_file_names = find_doc_files_in_current_folder()
print(list_file_names)

word_file_name = 'test close caption.docx'
doc = Document(word_file_name)


file_analysis_dict ={}
discussion_number = 1 #! number of speaker

file_speech_per_person = []
tmp_per_person_per_speech = []
rgl_time_pt = r'\b\d+:\d+\s'
ignore_header_count = 0

for paragraph in doc.paragraphs:
    text = paragraph.text
    
    text_detail = text.split(">>")
    
    if text_detail != ['']:

        if ignore_header_count < 2:
            #! this will help to get ride of the title and youtube linke
            ignore_header_count += 1

        elif len(text_detail) == 1 and match_pattern(rgl_time_pt, text=text_detail[0]):
            #! this will find the latest time stamp in the file
            temp_time = text_detail[0]
            # print(temp_time)

        elif len(text_detail) == 1 :
            tmp_per_person_per_speech.append(text_detail[0])

        elif len(text_detail) == 2 and  text_detail[0] == '':
            if tmp_per_person_per_speech != []:
                
                file_analysis_dict[discussion_number] = {
                    'end_time': temp_time,
                    'character_lenght': len(''.join(tmp_per_person_per_speech)),
                    'text': ''.join(tmp_per_person_per_speech),
                }
                discussion_number+=1

                file_speech_per_person.append(tmp_per_person_per_speech)
            
            tmp_per_person_per_speech=[]
            tmp_per_person_per_speech.append(text_detail[1])

        elif len(text_detail) == 2:
            tmp_per_person_per_speech.append(text_detail[0])
            
            file_analysis_dict[discussion_number] = {
                    'end_time': temp_time,
                    'character_lenght': len(''.join(tmp_per_person_per_speech)),
                    'text': ''.join(tmp_per_person_per_speech),
                }
            discussion_number+=1

            file_speech_per_person.append(tmp_per_person_per_speech)
            tmp_per_person_per_speech=[]
            tmp_per_person_per_speech.append(text_detail[1])

        elif len(text_detail) > 2:
            
            for indx in range(len(text_detail)):

                if indx == 0 and text_detail[indx] == ['']:

                    file_analysis_dict[discussion_number] = {
                    'end_time': temp_time,
                    'character_lenght': len(''.join(tmp_per_person_per_speech)),
                    'text': ''.join(tmp_per_person_per_speech),
                    }
                    discussion_number+=1

                    file_speech_per_person.append(tmp_per_person_per_speech)
                    tmp_per_person_per_speech=[]

                elif len(text_detail) == indx+1 :
                    tmp_per_person_per_speech.append(text_detail[indx])

                else:
                    tmp_per_person_per_speech.append(text_detail[indx])

                    file_analysis_dict[discussion_number] = {
                    'end_time': temp_time,
                    'character_lenght': len(''.join(tmp_per_person_per_speech)),
                    'text': ''.join(tmp_per_person_per_speech),
                    }
                    discussion_number+=1

                    file_speech_per_person.append(tmp_per_person_per_speech)
                    tmp_per_person_per_speech=[]


file_analysis_dict[discussion_number] = {
    'end_time': temp_time,
    'character_lenght': len(''.join(tmp_per_person_per_speech)),
    'text': ''.join(tmp_per_person_per_speech),
    }
discussion_number+=1

file_speech_per_person.append(tmp_per_person_per_speech)

exel_row_list = []
for item in file_analysis_dict:
    
    if item ==1:
        file_analysis_dict[item]['time_delta'] = calculate_time_delta('0:00', file_analysis_dict[item]['end_time'])
        file_analysis_dict[item]['start_time'] = '0:00'
        exel_row_list.append(file_analysis_dict[item])
    else:
        file_analysis_dict[item]['time_delta'] = calculate_time_delta(file_analysis_dict[item-1]['end_time'], file_analysis_dict[item]['end_time'])
        file_analysis_dict[item]['start_time'] = file_analysis_dict[item-1]['end_time']
        exel_row_list.append(file_analysis_dict[item])

    print(file_analysis_dict[item])


df = pd.DataFrame(exel_row_list)
excel_file_name =  word_file_name.replace('docx', 'xlsx')  #'data.xlsx'  # Change the file name if needed
df.to_excel(excel_file_name, index=False)

['June 2019 ACIP Meeting - Votes.docx', 'June 2019 ACIP Meeting - Combination Vaccines; Public Comment.docx', 'June 2019 ACIP Generated transcript from audio.docx']
{'end_time': '0:24 ', 'character_lenght': 365, 'text': " Good morning and welcome everyone to the second day of our ACIP meeting today is October 26. And I'm pleased to call our ACIP Meeting to order. I will now take roll call for our members only. As a reminder, we are going to ask that you state your conflicts because we do have a vote later today. And I will go in alphabetical order. So we'll start with Ms. Bahta. ", 'time_delta': 24, 'start_time': '0:00'}
{'end_time': '0:32 ', 'character_lenght': 122, 'text': ' Good morning, Lynn Bahta immunization clinical consultant at the Minnesota Department of Health and I have no conflicts. ', 'time_delta': 8, 'start_time': '0:24 '}
{'end_time': '0:32 ', 'character_lenght': 22, 'text': ' Thank you, Dr. Bell. ', 'time_delta': 0, 'start_time': '0:32 '}
{'end_time': '0:43 ', 'charact

In [25]:
rgl_time_pt = "\['(?:\d+:)?\d+:\d+'\]"

In [26]:
def calculate_time_delta(time1, time2):
    time1_parts = list(map(int, time1.split(':')))
    time2_parts = list(map(int, time2.split(':')))

    while len(time1_parts) < 3:
        time1_parts.insert(0, 0)
    while len(time2_parts) < 3:
        time2_parts.insert(0, 0)

    hours_diff = time2_parts[0] - time1_parts[0]
    minutes_diff = time2_parts[1] - time1_parts[1]
    seconds_diff = time2_parts[2] - time1_parts[2]

    total_seconds = hours_diff * 3600 + minutes_diff * 60 + seconds_diff
    return total_seconds

time1 = '00:10'
time2 = '2:00:00'

result = calculate_time_delta(time1, time2)
print("Delta:", result)  # Output: Delta: 1547


Delta: 7190
